In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch.utils.data 

from torchvision.models import *
from PIL import Image, ImageOps
import os
import os.path
import sys
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
import cv2
from sklearn import preprocessing
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from fastai import *
from fastai.vision import *
from fastai.layers import MSELossFlat, CrossEntropyFlat


In [ ]:
image_ids = []
xs = []
ys = []
zs = []
#c = 0
for dir, sub_dir, f in os.walk("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/train/sequences"):
  if len(f) > 0:
   # c += 1
    f = sorted(f)
    seq_id = dir.split("/")[-1]
    temp_images_ids = [seq_id+'_'+x for x in f]
    image_ids.extend(temp_images_ids)
    with open("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/train/labels/"+seq_id+".txt", mode = "r") as file:
      labels = file.read()
      labels = labels.split('\n')
      for line in labels:
        line = line.strip('\n')
        if len(line.split(',')) == 4:
          file_reqd, temp_x, temp_y, temp_z = line.split(',') 
          if file_reqd+'.png' in f:
            xs.append(float(temp_x))
            ys.append(float(temp_y))
            zs.append(float(temp_z))
  #if(c == 900):
  #    break

    
import pandas as pd

df = pd.DataFrame({"unique_id":image_ids,'x':xs,'y':ys,'z':zs})
df.head()


,unique_id,x,y,z
0,6679_000.png,-0.133114,-0.021778,0.990861
1,6679_001.png,-0.127335,-0.020104,0.991656
2,6679_002.png,-0.133111,-0.022769,0.990839
3,6679_003.png,-0.127335,-0.020104,0.991656
4,6679_005.png,-0.126190,-0.019146,0.991821


In [ ]:
df

,unique_id,x,y,z
0,6679_000.png,-0.133114,-0.021778,0.990861
1,6679_001.png,-0.127335,-0.020104,0.991656
2,6679_002.png,-0.133111,-0.022769,0.990839
3,6679_003.png,-0.127335,-0.020104,0.991656
4,6679_005.png,-0.126190,-0.019146,0.991821
...,...,...,...,...
80434,6681_092.png,-0.302105,0.234002,0.924108
80435,6681_094.png,-0.303538,0.234846,0.923424
80436,6681_095.png,-0.304603,0.234025,0.923282
80437,6681_096.png,-0.305366,0.233965,0.923045


In [ ]:
image_ids = []
xs = []
ys = []
zs = []
#c = 0
for dir, sub_dir, f in os.walk("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/validation/sequences"):
  if len(f) > 0:
    #c += 1
    f = sorted(f)
    seq_id = dir.split("/")[-1]
    temp_images_ids = [seq_id+'_'+x for x in f]
    image_ids.extend(temp_images_ids)
    with open("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/validation/labels/"+seq_id+".txt", mode = "r") as file:
      labels = file.read()
      labels = labels.split('\n')
      for line in labels:
        line = line.strip('\n')
        if len(line.split(',')) == 4:
          file_reqd, temp_x, temp_y, temp_z = line.split(',') 
          if file_reqd+'.png' in f:
            xs.append(float(temp_x))
            ys.append(float(temp_y))
            zs.append(float(temp_z))
    a = len(image_ids)
    while(a > len(xs)):
      image_ids.pop()
      a -= 1
  #if(c == 900):
    #break

import pandas as pd

df1 = pd.DataFrame({"unique_id":image_ids,'x':xs,'y':ys,'z':zs})
df1.head()


,unique_id,x,y,z
0,7960_000.png,0.204384,0.111141,0.972561
1,7960_001.png,0.203746,0.110971,0.972714
2,7960_002.png,0.202203,0.110214,0.973122
3,7960_003.png,0.201722,0.110209,0.973223
4,7960_004.png,0.201057,0.110906,0.973281


In [ ]:
df1

,unique_id,x,y,z
0,7960_000.png,0.204384,0.111141,0.972561
1,7960_001.png,0.203746,0.110971,0.972714
2,7960_002.png,0.202203,0.110214,0.973122
3,7960_003.png,0.201722,0.110209,0.973223
4,7960_004.png,0.201057,0.110906,0.973281
...,...,...,...,...
30007,8880_050.png,0.294119,0.106741,0.949790
30008,8880_051.png,0.294027,0.106727,0.949820
30009,8880_052.png,0.294054,0.107096,0.949770
30010,8880_053.png,0.294131,0.107340,0.949719


INFERENCES

accuracy

In [ ]:
from tqdm import tqdm

Gaze_X = []
Gaze_Y = []

with open('/content/drive/My Drive/FYP/Notebooks/xy.txt','r') as file:
  labels = file.read()
  labels = labels.split('\n')
  for line in tqdm(labels):
    line = line.strip('\n')
    if len(line.split(',')) == 2:
      temp_x, temp_y = line.split(',') 
      Gaze_X.append(float(temp_x))
      Gaze_Y.append(float(temp_y))

In [ ]:
Gaze_Z = []

with open('/content/drive/My Drive/FYP/Notebooks/z.txt','r') as file:
  for line in file:
    for temp_z in line.split():
      Gaze_Z.append(float(temp_z))

In [ ]:
xpixels = 0
ypixels = 0
offset = 0
mu = 0.28

In [ ]:
from tqdm import tqdm

avg_angacc = 0
for idx in tqdm(range(len(Gaze_X))):
  GazeX = Gaze_X[idx]
  GazeY = Gaze_Y[idx]
  Z = Gaze_Z[idx]
  GT_x = df1.iloc[idx].x
  GT_y = df1.iloc[idx].y
  OSD = mu * math.sqrt(pow((GazeX - xpixels/2),2) + pow((ypixels - GazeY + offset),2))
  gazeangle = math.atan(OSD/Z)
  EGP = math.sqrt(pow(GazeX, 2) + pow(GazeY, 2) + pow(Z, 2))
  pix_dist = math.sqrt(pow((GT_x - GazeX), 2) + pow((GT_y - GazeY), 2))
  angacc = (mu * pix_dist * pow(math.cos(gazeangle), 2))/EGP
  #print("Angular accuracy = ", angacc)
  avg_angacc += angacc

print("\n Average angular accuracy = ", avg_angacc/len(Gaze_X))


 Average angular accuracy =  0.0853466


In [ ]:
print(math.degrees(avg_angacc/len(Gaze_X)))

4.889999975791231
